In [0]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (2).csv


In [0]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,Id,Review,Score
0,1,\n So this is what it takes bands nowadays to ...,0.0
1,2,\nTotal Strife Forever is an eclectic electron...,3.0
2,3,\n Sonically this is closest to Isolation Dril...,2.0
3,4,\n Informative and representative of the time....,4.0
4,5,"\n Things like ""streamlined"" and ""accessible"" ...",2.0


In [0]:
df1 = pd.DataFrame({'id':df['Id'],
            'Review':df['Review'].replace(r'\n',' ',regex=True),
            'Score':df['Score']})

In [0]:
import numpy as np # linear algebra

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
num_of_categories = 4000
shuffled = df.reindex(np.random.permutation(df.index))
e = shuffled[shuffled['Score'] == 0.0][:num_of_categories]
b = shuffled[shuffled['Score'] == 1.0][:num_of_categories]
t = shuffled[shuffled['Score'] == 2.0][:num_of_categories]
m = shuffled[shuffled['Score'] == 3.0][:num_of_categories]
j = shuffled[shuffled['Score'] == 4.0][:num_of_categories]
concated = pd.concat([e,b,t,m,j], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))

In [0]:
concated.shape

(14623, 3)

In [0]:
n_most_common_words = 8000
max_len = 200
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['Review'].values)
sequences = tokenizer.texts_to_sequences(concated['Review'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 44311 unique tokens.


In [0]:
y1 = concated['Score']
from sklearn import preprocessing
import keras
le = preprocessing.LabelEncoder()
le.fit(y1)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)
def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)



Y = encode(le, y1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X , Y, test_size=0.25, random_state=42)

In [0]:
epochs = 10
emb_dim = 128
batch_size = 256
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

In [0]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

5156/5156 [==============================] - 38s 7ms/step
Test set
  Loss: 1.166
  Accuracy: 0.510
